# 3. Timestamps, TTLs and Events 
### Converted to seconds and saved

In [ ]:
import spikeinterface.full as si
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import os
import csv
import glob
from datetime import datetime
import warnings
warnings.simplefilter("ignore")
%matplotlib widget
print(f"SpikeInterface Version: {si.__version__}")

In [ ]:
# Setting file paths and basic parameters
base_folder = Path('D:/Ephys_C2DRG/')
data_folder = Path("D:/Ephys_C2DRG/2023_9_19/")

In [ ]:
# Function to calculate time difference in seconds
def time_difference_in_seconds(start_time, end_time):
    start = datetime.combine(datetime.min, start_time)
    end = datetime.combine(datetime.min, end_time)
    return (end - start).total_seconds()

# Load the Timestamps CSV files, extract start time
timestamp_start_df = pd.read_csv(data_folder / 'TimestampsEphys_0.csv', nrows=1, header=None, names=['Timestamps'])
timestamp_start = pd.to_datetime(timestamp_start_df['Timestamps'][0]).time() # Convert timestamps to datetime objects (only time part)

#Load times from timestamps csv files and calculate start time in seconds.
tms_files = sorted(glob.glob(os.path.join(data_folder, "Timestamps*.csv")))
concatenated_segment_times = pd.DataFrame()
for tms_file in tms_files:
    tms_df = pd.read_csv(tms_file, header=None)#, names=['Start_Times', 'End_Times'])#(usecols=[0], nrows=1)
    segment_times = pd.DataFrame({'Start_Times': tms_df.iloc[0], 'End_Times': tms_df.iloc[-1]})
    concatenated_segment_times = pd.concat([concatenated_segment_times, segment_times], ignore_index=True)
concatenated_segment_times['Start_Times'] = pd.to_datetime(concatenated_segment_times['Start_Times']).dt.time # Convert the tms timestamps to timedelta (ignoring date)
concatenated_segment_times['End_Times'] = pd.to_datetime(concatenated_segment_times['End_Times']).dt.time # Convert the tms timestamps to timedelta (ignoring date)
# Apply the time difference function to each row in the tms DataFrame
concatenated_segment_times['Start_Times_seconds'] = concatenated_segment_times['Start_Times'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
concatenated_segment_times['End_Times_seconds'] = concatenated_segment_times['End_Times'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
concatenated_segment_times['Segment_duration_seconds'] = concatenated_segment_times['End_Times_seconds'] - concatenated_segment_times['Start_Times_seconds']
#concatenated_segment_times.to_csv(data_folder / 'concatenated_segment_times.csv', mode='x', index=False)
print(concatenated_segment_times)

In [ ]:
#Load ttl times from csv files, calculate time in seconds
ttl_files = sorted(glob.glob(os.path.join(data_folder, "TTL*.csv")))
concatenated_ttl_times = pd.DataFrame()
for ttl_file in ttl_files:
    TTL_df = pd.read_csv(ttl_file, header=None, names=['TTL_Times'])#(usecols=[0], nrows=1)
    concatenated_ttl_times = pd.concat([concatenated_ttl_times, TTL_df], ignore_index=True)
concatenated_ttl_times['TTL_Times'] = pd.to_datetime(concatenated_ttl_times['TTL_Times']).dt.time # Convert the TTL timestamps to timedelta (ignoring date)
# Apply the time difference function to each row in the TTL DataFrame
concatenated_ttl_times['time_diff_seconds'] = concatenated_ttl_times['TTL_Times'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
#concatenated_ttl_times.to_csv(data_folder / 'concatenated_ttl_times.csv', mode='x', index=False)
print(concatenated_ttl_times)

#Load events times from csv files, calculate time in seconds
events_files = sorted(glob.glob(os.path.join(data_folder, "Events*.csv")))
concatenated_event_times = pd.DataFrame()
for event_file in events_files:
    Events_df = pd.read_csv(event_file, header=None, usecols=[0, 1], names=['Stim_start', 'Stim_end'])#(usecols=[0], nrows=1)
    concatenated_event_times = pd.concat([concatenated_event_times, Events_df], ignore_index=True)
concatenated_event_times['Stim_start'] = pd.to_datetime(concatenated_event_times['Stim_start']).dt.time # Convert the TTL timestamps to timedelta (ignoring date)
concatenated_event_times['Stim_end'] = pd.to_datetime(concatenated_event_times['Stim_end']).dt.time # Convert the TTL timestamps to timedelta (ignoring date)
# Apply the time difference function to each row in the TTL DataFrame
concatenated_event_times['time_diff_start_seconds'] = concatenated_event_times['Stim_start'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
concatenated_event_times['time_diff_end_seconds'] = concatenated_event_times['Stim_end'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
concatenated_event_times['stim_duration'] = concatenated_event_times['time_diff_end_seconds'] - concatenated_event_times['time_diff_start_seconds']
#concatenated_event_times.to_csv(data_folder / 'concatenated_event_times.csv', mode='x', index=False) 
print(concatenated_event_times)